In [2]:
import Pkg

Pkg.add("HomotopyContinuation")
Pkg.add("Plots")
Pkg.add("PyPlot")
Pkg.add("DataFrames")
Pkg.add("CSV")
Pkg.add("SymPy")
# Pkg.add("Symbolics")

    Updating registry at `~/.julia/registries/General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environ

In [2]:
VERSION

v"1.7.1"

In [3]:
using HomotopyContinuation
using Plots
using DelimitedFiles
using DataFrames
using CSV
using SymPy
# using Symbolics

In [4]:
function get_roots(f)
    @var y
    roots = []
    # define the polynomials
    F = System([f])
    result = HomotopyContinuation.solve(F)
    for root in results(result)
        push!(roots, root.solution[1])
    end
    return roots
end

@var m
# get_roots(m^2 + 1/2)
get_roots(m^2 + 1/2)

2-element Vector{Any}:
 2.2420775429197073e-44 + 0.7071067811865476im
 2.2420775429197073e-44 - 0.7071067811865475im

In [41]:
h = 0.0001
# n=5
num = 15


@var t
n = (4*(t^num))
d = (t^num + 1)^2








(1 + t^15)^2

In [42]:
x = symbols("x") 
belyi =   (4*(x^num)) / (x^num + 1)^2
# diff(n/d)
     # PyObject x
# SymPy.diff(n)
typeof(n)
# ex = diff(belyi)


body = convert(Expr, belyi)
syms = Symbol.(free_symbols(belyi))
bt = eval(Expr(:function, Expr(:call, gensym(), syms...), body))

ex = diff(belyi)
bodydiff = convert(Expr, ex)
symsdiff = Symbol.(free_symbols(ex))
dt = eval(Expr(:function, Expr(:call, gensym(), syms...), bodydiff))
SymPy.degree(belyi)

15

In [43]:




# bt(t) = (4*(t^n)) / (t^n + 1)^2
# dt(t) = -1 * (((4 * n * (t^(n-1)) * (t^n - 1))) / ((t^n+1)^3))


In [44]:
function stereographic(c)
    x = (2 * real(c)) / (abs(c) ^ 2 + 1)
    y = (2 * imag(c)) / (abs(c) ^ 2 + 1)
    z = (abs(c)^2 - 1) / (abs(c)^2 + 1)
    return (x, y, z)
end

stereographic (generic function with 1 method)

In [65]:
function rk4(start, stop, initialQ, initialE)
    points = []
    q = initialQ
    e = initialE
    push!(points, e)
    while q < stop
        nexth = abs(dt(e)) * h
        nextq = q + nexth
        
        
        k1 = nexth * (1/dt(e))
        k2 = nexth * (1/dt(e + k1/2))
        k3 = nexth * (1/dt(e + k2/2))
        k4 = nexth * (1/dt(e + k3))
        
        nexte = e + (1/6) * (k1 + 2*k2 + 2*k3 + k4)
        push!(points, nexte)
        q = nextq

        
        e = nexte
    end
    q = initialQ
    e = initialE
    
    while q > .00000000000000000001
        nexth = abs(dt(e)) * h
        nextq = q - nexth
        
        k1 = nexth * (1/dt(e))
        k2 = nexth * (1/dt(e - k1/2))
        k3 = nexth * (1/dt(e - k2/2))
        k4 = nexth * (1/dt(e - k3))
        
        nexte = e - (1/6) * (k1 + 2*k2 + 2*k3 + k4)
        push!(points, nexte)
        q = nextq
        e = nexte
    end
    return points
end

@var x
points = []

roots = get_roots(n - 1/2 * d)
print("ROOTS FOUND: $(length(roots))")

for root in roots

    edge = rk4(0, 1, 0.5, root)
    points = vcat(points, edge)
end

ROOTS FOUND: 30

In [ ]:

xs = []
ys = []
zs = []
for point in points
    x, y, z = stereographic(point)
    x = round(x, digits=3)
    y = round(y, digits=3)
    z = round(z, digits=3)
    push!(xs, x)
    push!(ys, y)
    push!(zs, z)
end

# df = DataFrame(X=xs, Y=ys, Z=zs)
# CSV.write("test.csv", df)

scatter(xs[1:1000:length(xs)], ys[1:1000:length(xs)], zs[1:1000:length(xs)])


In [72]:
sampling = 1000
l = length(xs)/sampling
io = open("juliaPCD.pcd", "w"); 
write(io,
"
# .PCD v0.7 - Point Cloud Data file format
VERSION 0.7
FIELDS x y z
SIZE 4 4 4
TYPE F F F
COUNT 1 1 1
WIDTH $l
HEIGHT 1
VIEWPOINT 0 0 0 1 0 0 0
POINTS $l
DATA ascii\n
")
    for i in 1:sampling:(length(xs))
        write(io, "$(xs[i]) $(ys[i]) $(zs[i])\n")
    end

close(io);


In [80]:
xs[1:2:length(xs)]

4134351-element Vector{Any}:
 0.993
 0.993
 0.993
 0.993
 0.993
 0.993
 0.993
 0.993
 0.993
 0.993
 0.993
 0.993
 0.993
 ⋮
 0.205
 0.205
 0.206
 0.206
 0.206
 0.207
 0.207
 0.207
 0.208
 0.208
 0.209
 0.209